In [15]:
%load_ext autoreload
%autoreload 2

import cv2
import matplotlib.pyplot as plt
import numpy as np
import time
from tqdm.auto import tqdm
from IPython.display import clear_output
from pathlib import Path

from PIL import Image

from loader import DataLoader
from registration.first_registration import Align
from models.ransacflow import register_image

DATA_DIR = "/Users/farhanoktavian/imperial/thesis/sandbox/dataset/raw"
METADATA_DIR = "/Users/farhanoktavian/imperial/thesis/sandbox/dataset/data.h5"
STACKREG_DIR = Path("/Users/farhanoktavian/imperial/thesis/sandbox/dataset/stackreg5")

run_loader = DataLoader(DATA_DIR, METADATA_DIR)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
sample = run_loader.samples[16]
run = sample.runs[0]
print(f"{sample.sample_id} - {run.run_id}")

HS017 - Run01


In [26]:
target_wavelength = 600

start_t = time.time()

for c in range(run.collection_count):
    print(f"Collection {c}/{run.collection_count}")
    imgs, img_paths = run.get_spectral_images(collection_idx=c, with_path=True)
    tgt_img = run.get_spectral_image(wavelength=target_wavelength, collection_idx=c)
    for i in tqdm(range(len(imgs))):
        img = imgs[i]
        img_path = img_paths[i]

        blank_img = np.zeros_like(img)
        temp_img = cv2.merge([img, blank_img, blank_img])

        blank_tgt_img = np.zeros_like(tgt_img)
        temp_tgt_img = cv2.merge([tgt_img, blank_tgt_img, blank_tgt_img])

        img = register_image(Image.fromarray(temp_img), Image.fromarray(temp_tgt_img))
        img = img[:, :, 0]

        save_path = STACKREG_DIR / sample.sample_id / run.run_id
        save_path.mkdir(parents=True, exist_ok=True)
        cv2.imwrite(str(save_path / img_path.name), img)

end_t = time.time()
print(f"Time taken: {end_t - start_t}s")

Collection 0/2


100%|██████████| 29/29 [01:59<00:00,  4.11s/it]


Collection 1/2


100%|██████████| 29/29 [02:04<00:00,  4.29s/it]

Time taken: 244.83328223228455s
